In [186]:
import pandas as pd
import numpy as np
import re

## ler arquivo json em pandas
df = pd.read_json('project-1-at-2023-04-27-03-26-58dd1708.json')
## expandir coluna annotations
df = pd.concat([df.drop(['annotations'], axis=1), df['annotations'].apply(pd.Series)], axis=1)
## expandir coluna 0 e renomear para annotations
df = pd.concat([df.drop([0], axis=1), df[0].apply(pd.Series)], axis=1)
## expandir coluna result e renomear para result
df = pd.concat([df.drop(['result'], axis=1), df['result'].apply(pd.Series)], axis=1)
## expandir coluna 0 e renomear para result
df = pd.concat([df.drop([0], axis=1), df[0].apply(pd.Series)], axis=1)
## expandir coluna value e renomear para value
df = pd.concat([df.drop(['value'], axis=1), df['value'].apply(pd.Series)], axis=1)
## dropar choices nulos
df = df.dropna(subset=['choices'])
## obter choices 
df['choices'] = df['choices'].apply(lambda x: x[0])


## expandir coluna data
df = pd.concat([df.drop(['data'], axis=1), df['data'].apply(pd.Series)], axis=1)
padrao_data_cepea = r"Cepea, \d{2}/\d{2}/\d{4} - "
df['noticia'] = df['noticia'].apply(lambda x: re.sub(padrao_data_cepea, '', x))

## conter apenas soja no titulo
#df = df[df['titulo'].str.contains('SOJA')]
columns_to_select = ['id', 'data', 'noticia', 'titulo', 'choices']
df = df[columns_to_select]
df_modelo = df[:122]
df_validacao = df[122:]



In [187]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix

In [188]:
X = df_modelo['noticia']
y = df_modelo['choices']

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)


nb = MultinomialNB()
nb.fit(X_train, y_train)

y_pred = nb.predict(X_test)
print(classification_report(y_test, y_pred, zero_division=0))


              precision    recall  f1-score   support

    Negativa       0.33      0.44      0.38         9
      Neutra       0.00      0.00      0.00         7
    Positiva       0.56      0.67      0.61        21

    accuracy                           0.49        37
   macro avg       0.30      0.37      0.33        37
weighted avg       0.40      0.49      0.44        37



In [213]:
## validar o modelo
X_validacao = df_validacao['noticia']
y_validacao = df_validacao['choices']

X_validacao = vectorizer.transform(X_validacao)

y_pred = nb.predict(X_validacao)

print(classification_report(y_validacao, y_pred, zero_division=0))

frase = 'mercado da soja em baixa'
frase = vectorizer.transform([frase])
nb.predict(frase)

# ## gerar dataframe com resultados
# df_resultado = pd.DataFrame({'noticia': df_validacao['noticia'], 'y_validacao': y_validacao, 'y_pred': y_pred})
# df_resultado

# ## salvar modelo
import pickle
pickle.dump(nb, open('modelo.pkl', 'wb'))
pickle.dump(vectorizer, open('vectorizer.pkl', 'wb'))


              precision    recall  f1-score   support

    Negativa       0.53      0.91      0.67        22
      Neutra       0.00      0.00      0.00        10
    Positiva       0.73      0.47      0.57        17

    accuracy                           0.57        49
   macro avg       0.42      0.46      0.41        49
weighted avg       0.49      0.57      0.50        49



In [215]:
## carregar modelo
modelo = pickle.load(open('modelo.pkl', 'rb'))
vectorizer = pickle.load(open('vectorizer.pkl', 'rb'))

## testar modelo
frase = 'mercado da soja em alta'
frase = vectorizer.transform([frase])
modelo.predict(frase)


array(['Positiva'], dtype='<U14')